In [1]:
###################################################################################################
# MultiSenseDataset file EXTRACTION for Automatic Targetless Lidar Camera Calibration-Repository  #
# https://github.com/xmba15/automatic_lidar_camera_calibration                                    #
###################################################################################################
%load_ext autoreload
%autoreload 2

import imageio
from PIL import Image
import open3d as o3d
import os
import json
from ipywidgets import IntProgress
from IPython.display import display
from scipy.spatial import transform
import sys
sys.path.insert(0, '../../datasets/')
from MultiSenseDataset import *
from PyKitti2Dataset import PyKitti2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
from AutoCalibration import  euler_diff, quat_diff
from scipy.spatial.transform import Rotation as Rot


def distort_calibration_(RT, range_t=[0.01, 0.01, 0.01], range_R=[2, 2, 2]):
        R, T = randomRT(range_t, range_R)
        RT[:3, 3] = RT[:3, 3] + T
        
        r = Rot.from_euler('zyx', R, degrees=True)
        
        r = r.as_matrix()
        
        RT[:3, :3] = r @ RT[:3, :3]
        
        return RT
        

def randomRT(range_t, range_R):
    T = np.random.uniform(-range_t, range_t, (3,))
    R = np.random.uniform(-range_R, range_R, (3,))

    if quat_diff(Rot.from_euler('zyx', R, degrees=True).as_matrix(), np.eye(3)) > range_R or np.linalg.norm(T) > range_t:
        return randomRT(range_t, range_R)

    if quat_diff(Rot.from_euler('zyx', R, degrees=True).as_matrix(), np.eye(3)) < range_R * 0.5 or np.linalg.norm(T) > range_t < range_R * 0.5:
        return randomRT(range_t, range_R)

    return R, T
    


# Dataset folder
#data_folder = '/home/ios/data3/multisense'
#data_folder = '/home/ios/data3/multisense_stereo' # stereo setup

# Destination folder
#dest_folder = "/home/ios/data3/multisense_gNMI_data/"
#dest_folder = "/home/ios/data3/multisense_gNMI_stereo_data/"

# Ground Truth
#gt_RT = "/home/ios/GitLab/rgbd-registration/rgb-lidar-reg/calib/multisense_seq_27_RT_old_to_new_recentered_GT.npy"
#gt_RT = "/home/ios/GitLab/rgbd-registration/rgb-lidar-reg/calib/multisense_seq_stereo_TR_icp_dist_to_new_improved_GT.npy"



ds = MultiSenseData(data_folder, rectified=True, crop=[0, 0, -1, -1])
# Make directories
try:
    os.mkdir(dest_folder + "images")
    print("Creating images directory")
except OSError as error:
    print("images directory already exists.")
try:
    os.mkdir(dest_folder + "point_clouds")
    print("Creating point_clouds directory")
except OSError as error:
    print("point_clouds directory already exists.")


# execute for loop
exec_loop = False

# fancy progress bar
f = IntProgress(min=0, max=len(ds))
display(f)

# for every sample in Dataset
if exec_loop:
    for i in range(len(ds)):
        f.value += 1
        f.description = "{:.2f}".format((i+1) / len(ds) * 100) + "%"

        # Write image
        img = ds[i][1]
        img.save(dest_folder + "images" + "/" + str(i) + ".jpg", 'JPEG')

        # Write PointCloud
        pc = ds[i][0][:, :3] # positions
        intensities = ds[i][0][:, 3] # intensities
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pc)

        intensities = np.array([intensities, [0]*len(intensities), [0]*len(intensities)]).T
        pcd.normals = o3d.utility.Vector3dVector(intensities)
        o3d.io.write_point_cloud(dest_folder + "/" + "point_clouds" + "/" + str(i) + ".pcd", pcd, write_ascii=True)

        # Replace FIELD "normal_x" with "intensity" in pcd file
        with open(dest_folder + "/" + "point_clouds" + "/" + str(i) + ".pcd") as file:
            newContent = file.read().replace('normal_x', 'intensity')
        with open(dest_folder + "/" + "point_clouds" + "/" + str(i) + ".pcd", "w") as file:
            file.write(newContent)
        
# Write images.txt (absolute file paths to images)
with open(dest_folder + "images.txt", 'a') as img_file:
    img_file.truncate(0) # erase content
    for i in range(10):
        img_file.write(dest_folder + "images" + "/" + str(i) + ".jpg\n")

# Write point_clouds.txt (absolute file paths to point clouds)
with open(dest_folder + "point_clouds.txt", 'a') as pc_file:
    pc_file.truncate(0) # erase content
    for i in range(10):
        pc_file.write(dest_folder + "point_clouds" + "/" + str(i) + ".pcd\n")


# Write K-Matrix (camera_info.json)
K = ds.calib.P_rect_20
K_JSON = { "K": [ K[0, 0], K[0, 1], K[0, 2],
                  K[1, 0], K[1, 1], K[1, 2],
                  K[2, 0], K[2, 1], K[2, 2]] }
with open(dest_folder + 'camera_info.json', 'w', encoding='utf-8') as f:
    json.dump(K_JSON, f, ensure_ascii=False, indent=4)
    
# Write Ground Truth to JSON file#
#distortions = [1e-8, 0.03, 0.05]
distortions = [1e-8, 0.02, 0.06]
for i in range(3):
    for j in range(5):
        GT = np.load(gt_RT)

        GT = distort_calibration_(GT.copy(), range_t=distortions[i], range_R=1.87)

        #print(euler_diff(GT_[:3, :3], GT[:3, :3]))

        rot = transform.Rotation.from_matrix(GT[:3, :3]).as_euler("zyx", degrees=True)
        GT_JSON = { "x": GT[0,3],
                    "y": GT[1,3],
                    "z": GT[2,3],
                    "r_deg": rot[2],
                    "p_deg": rot[1],
                    "y_deg": rot[0] }
        with open(dest_folder + f'initial_guess_{i}_{j}.json', 'w', encoding='utf-8') as f:
            json.dump(GT_JSON, f, ensure_ascii=False, indent=4)


print("Done.")

IndexError: list index out of range

In [5]:
################################
# Same stuff for KITTI dataset #
################################
#%cd ../..
from AutoCalibration import  euler_diff, quat_diff
from scipy.spatial.transform import Rotation as Rot


def distort_calibration_(RT, range_t=[0.01, 0.01, 0.01], range_R=[2, 2, 2]):
        R, T = randomRT(range_t, range_R)
        RT[:3, 3] = RT[:3, 3] + T
        
        r = Rot.from_euler('zyx', R, degrees=True)
        
        r = r.as_matrix()
        
        RT[:3, :3] = r @ RT[:3, :3]
        
        return RT
        

def randomRT(range_t, range_R):
    T = np.random.uniform(-range_t, range_t, (3,))
    R = np.random.uniform(-range_R, range_R, (3,))

    if quat_diff(Rot.from_euler('zyx', R, degrees=True).as_matrix(), np.eye(3)) > range_R or np.linalg.norm(T) > range_t:
        return randomRT(range_t, range_R)

    if quat_diff(Rot.from_euler('zyx', R, degrees=True).as_matrix(), np.eye(3)) < range_R * 0.5 or np.linalg.norm(T) > range_t < range_R * 0.5:
        return randomRT(range_t, range_R)

    return R, T
    

# Whether or not to convert bin-PointCloud to pcl-File    
convert_pc = False



for i in range(10, 14): # sequence 10 to 13
    # Dataset folder
    data_folder = "/home/ios/data3/kitti/tracking/training/"

    # Destination folder
    dest_folder = '/home/ios/data3/kitti/tracking/training/gNMI/'

    # Dataset
    seq = f"00{i}"
    ds = PyKitti2(data_folder, seq, with_labels="mask")
    
    # Ground Truth
    gt_RT = ds.calib.T_cam2_velo
    
    if convert_pc:
        for x in range(1):
            # Write PointCloud
            pc = ds[x][0].copy()[:, :3] # positions
            
            intensities = ds[x][0].copy()[:, 3] # intensities
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(pc)

            intensities = np.array([intensities, [0]*len(intensities), [0]*len(intensities)]).T
            
            intensities = ( intensities - intensities.min()) / ( intensities.max() - intensities.min() + 1e-8) * 101 
            pcd.normals = o3d.utility.Vector3dVector(intensities)
            o3d.io.write_point_cloud("/home/ios/data3/kitti/tracking/training/velodyne_pcl" + "/" + f"00{i}" + "/" + str(x).zfill(6) + ".pcd", pcd, write_ascii=True)
            # Replace FIELD "normal_x" with "intensity" in pcd file
            with open("/home/ios/data3/kitti/tracking/training/velodyne_pcl" + "/" + f"00{i}" + "/" + str(x).zfill(6) + ".pcd", "r") as file:
                newContent = file.read().replace('normal_x', 'intensity')
            with open("/home/ios/data3/kitti/tracking/training/velodyne_pcl" + "/" + f"00{i}" + "/" + str(x).zfill(6) + ".pcd", "w") as file:
                file.write(newContent)
    
    

    # Write images.txt (absolute file paths to images)
    with open(dest_folder + f"/images_{i}.txt", 'a') as img_file:
        img_file.truncate(0) # erase content
        files_list = os.listdir(f"/home/ios/data3/kitti/tracking/training/image_02/00{i}/")
        files_list.sort()
        for k in files_list:
            img_file.write(f"/home/ios/data3/kitti/tracking/training/image_02/00{i}/" + k + "\n")

    # Write point_clouds.txt (absolute file paths to point clouds)
    with open(dest_folder + f"/point_clouds_{i}.txt", 'a') as pc_file:
        pc_file.truncate(0) # erase content
        files_list = os.listdir(f"/home/ios/data3/kitti/tracking/training/velodyne_pcl/00{i}/")
        files_list.sort()
        for k in files_list:
            pc_file.write(f"/home/ios/data3/kitti/tracking/training/velodyne_pcl/00{i}/" + k + "\n")

    # Write Ground Truth to JSON file
    distortion = 0.06#0.06
    for j in range(5):
        GT = gt_RT.copy()

        GT_ = distort_calibration_(GT.copy(), range_t=distortion, range_R=1.87)

        rot = transform.Rotation.from_matrix(GT_[:3, :3]).as_euler("zyx", degrees=True)
        print("rot")
        print(rot)
        print("euler diff:")
        print(euler_diff(GT[:3, :3], GT_[:3, :3]))
        print()
        GT_JSON = { "x": GT_[0,3],
                    "y": GT_[1,3],
                    "z": GT_[2,3],
                    "r_deg": rot[2],
                    "p_deg": rot[1],
                    "y_deg": rot[0] }
        with open(dest_folder + f'initial_guess_{i}_{j}.json', 'w', encoding='utf-8') as f:
            json.dump(GT_JSON, f, ensure_ascii=False, indent=4)

        
# Write K-Matrix (camera_info.json)
K = ds.calib.P_rect_20
K_JSON = { "K": [ K[0, 0], K[0, 1], K[0, 2],
                  K[1, 0], K[1, 1], K[1, 2],
                  K[2, 0], K[2, 1], K[2, 2]] }
with open(dest_folder + 'camera_info.json', 'w', encoding='utf-8') as f:
    json.dump(K_JSON, f, ensure_ascii=False, indent=4)


print("Done.")

files 294


/home/ios/anaconda3/envs/lccnet/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


rot
[90.28313272 -1.11098152 90.55227106]
euler diff:
[ 1.14798433 -0.29168025 -0.50855311]

rot
[ 9.16143421e+01 -5.34878315e-02  9.01072803e+01]
euler diff:
[ 0.68897948 -1.62873425  0.54896244]

rot
[89.00448793 -1.33014694 89.12566491]
euler diff:
[-0.26515642  0.97083097 -0.73970349]

rot
[91.37165669 -1.66557276 89.11183638]
euler diff:
[-0.30405107 -1.40129736 -1.03854351]

rot
[9.04101135e+01 8.92404419e-02 9.08207726e+01]
euler diff:
[ 1.41512203 -0.43344713  0.68837422]

files 373
rot
[90.27866619  1.01671742 89.7857999 ]
euler diff:
[ 0.3816565  -0.28603725  1.62305818]

rot
[89.8805646  -1.5630495  89.74419122]
euler diff:
[ 0.34420533  0.10170019 -0.95825678]

rot
[88.85770945 -1.28517643 88.40396938]
euler diff:
[-0.98543228  1.10940376 -0.71098078]

rot
[90.63230938  0.49298325 89.77021624]
euler diff:
[ 0.36228154 -0.64131616  1.10085479]

rot
[89.89341023  0.37814275 90.39962588]
euler diff:
[0.99941191 0.08627141 0.98402165]

files 78
rot
[89.22382344  0.55052609 90.6

In [15]:
%pwd

'/home/ios/GitLab/rgbd-registration/rgb-lidar-reg'